In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 0 #当日は0、前日は1・・・
date2 = "08/19"
pmc = "27084001" # 店舗ID
SPREADSHEET_KEY = '1Z0Z1IDgbj-mGcRZxXEFpGlO00LHMQB_X5qll6QXUEOQ' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 514 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+789@gmail.com"
user_pw = "paruparu23"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
chrome_service = ChromeService(executable_path='C:\\Users\\garden026\\WD\\114.0.5735.90\\chromedriver.exe')

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
s20row = worksheet.cell(11,  28).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
s20table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2127":
                    s20table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2127":
                        s20table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})

['08/19', '025877', '1', '43077.0', '-6018.0', '36', '18']
['08/19', '025877', '2', '47692.0', '11436.0', '66', '20']
['08/19', '025877', '3', '32564.0', '3913.0', '39', '11']
['08/19', '025877', '4', '41795.0', '1506.0', '47', '17']
['08/19', '025877', '5', '31538.0', '10834.0', '47', '15']
['08/19', '025877', '6', '28462.0', '2408.0', '33', '10']
['08/19', '025877', '7', '41795.0', '13844.0', '63', '22']
['08/19', '025877', '8', '38718.0', '-3911.0', '36', '15']
['08/19', '025877', '9', '43333.0', '12640.0', '62', '22']
['08/19', '025877', '10', '40513.0', '2107.0', '46', '23']
['08/19', '025877', '11', '46923.0', '1506.0', '53', '18']
['08/19', '025877', '12', '43590.0', '-6619.0', '37', '14']
['08/19', '025877', '13', '43846.0', '-1504.0', '45', '24']
['08/19', '025877', '14', '41026.0', '12339.0', '60', '18']
['08/19', '025877', '15', '34359.0', '-3911.0', '36', '16']
['08/19', '025877', '16', '24359.0', '-2106.0', '24', '14']
['08/19', '025877', '17', '42564.0', '9931.0', '55', '

['08/19', '025856', '151', '35128.0', '-25277.0', '3', '1']
['08/19', '025856', '152', '42564.0', '-19860.0', '11', '6']
['08/19', '025856', '153', '23077.0', '22570.0', '28', '2']
['08/19', '025892', '154', '17949.0', '2408.0', '25', '8']
['08/19', '025892', '155', '33846.0', '10533.0', '49', '17']
['08/19', '025544', '158', '23077.0', '-9930.0', '30', '7']
['08/19', '025544', '159', '15897.0', '7223.0', '72', '7']
['08/19', '025865', '160', '13846.0', '8126.0', '64', '4']
['08/19', '025761', '161', '13077.0', '-8425.0', '10', '2']
['08/19', '025715', '162', '8718.0', '-4513.0', '7', '7']
['08/19', '025780', '163', '12051.0', '-2707.0', '19', '7']
['08/19', '025647', '164', '17692.0', '-7522.0', '21', '8']
['08/19', '025835', '165', '12564.0', '-8124.0', '16', '8']
['08/19', '025575', '166', '14872.0', '-5717.0', '14', '10']
['08/19', '025865', '167', '19231.0', '-11735.0', '14', '6']
['08/19', '025891', '168', '18718.0', '-6920.0', '22', '15']
['08/19', '025885', '169', '16154.0', '-

['08/19', '025818', '291', '19487.0', '-8726.0', '7', '3']
['08/19', '025818', '292', '30513.0', '24978.0', '36', '7']
['08/19', '025665', '293', '46923.0', '-14444.0', '18', '6']
['08/19', '025665', '294', '40256.0', '18658.0', '36', '10']
['08/19', '025665', '295', '44359.0', '2709.0', '28', '7']
['08/19', '025665', '296', '42051.0', '-9328.0', '19', '5']
['08/19', '025665', '297', '37949.0', '-24675.0', '8', '7']
['08/19', '025665', '298', '37949.0', '43033.0', '51', '8']
['08/19', '025665', '299', '36410.0', '3010.0', '23', '3']
['08/19', '025665', '300', '40769.0', '49955.0', '58', '10']
['08/19', '025665', '301', '0', '0', '0', '0']
['08/19', '025665', '302', '35385.0', '6019.0', '25', '6']
['08/19', '025665', '303', '41795.0', '10533.0', '31', '5']
['08/19', '025665', '304', '45385.0', '-7221.0', '22', '7']
['08/19', '025665', '305', '37692.0', '-20462.0', '8', '4']
['08/19', '025665', '306', '45897.0', '17455.0', '38', '8']
['08/19', '025665', '307', '38205.0', '27385.0', '40',

['08/19', '025489', '442', '22564.0', '904.0', '9', '3']
['08/19', '025489', '443', '9487.0', '-1203.0', '4', '3']
['08/19', '025489', '444', '35897.0', '-1504.0', '13', '3']
['08/19', '025489', '445', '9231.0', '-1203.0', '3', '2']
['08/19', '025489', '446', '20513.0', '9931.0', '14', '2']
['08/19', '025489', '447', '21795.0', '-9930.0', '3', '3']
['08/19', '025489', '448', '20256.0', '14144.0', '16', '2']
['08/19', '025489', '449', '31026.0', '-10832.0', '7', '5']
['08/19', '025557', '452', '40513.0', '3913.0', '32', '10']
['08/19', '025557', '453', '18462.0', '-2707.0', '10', '4']
['08/19', '025557', '454', '45385.0', '-2707.0', '27', '10']
['08/19', '025557', '455', '16923.0', '-3309.0', '8', '1']
['08/19', '025557', '456', '21026.0', '-6319.0', '7', '1']
['08/19', '025879', '457', '16923.0', '603.0', '16', '5']
['08/19', '025879', '458', '22051.0', '15348.0', '35', '5']
['08/19', '025553', '459', '25897.0', '-3911.0', '20', '3']
['08/19', '025553', '460', '24615.0', '-1203.0', '19

['08/19', '119916', '587', '2557', '-787.0', '7', '5', '--']
['08/19', '119916', '588', '6850', '509.0', '28', '17', '--']
['08/19', '119916', '589', '8151', '4306.0', '47', '21', '--']
['08/19', '119916', '590', '4675', '1019.0', '24', '7', '--']
['08/19', '120010', '591', '5187', '-93.0', '18', '12', '--']
['08/19', '120010', '592', '7400', '1204.0', '34', '19', '--']
['08/19', '120010', '593', '5533', '-2083.0', '14', '13', '--']
['08/19', '120010', '594', '6359', '648.0', '28', '16', '--']
['08/19', '120010', '595', '3047', '-417.0', '10', '7', '--']
['08/19', '120010', '596', '7104', '1991.0', '35', '21', '--']
['08/19', '120010', '597', '7826', '1806.0', '34', '30', '--']
['08/19', '120010', '598', '7688', '-602.0', '26', '22', '--']
['08/19', '120010', '599', '7019', '972.0', '30', '22', '--']
['08/19', '120010', '600', '2726', '-1065.0', '6', '6', '--']
['08/19', '120010', '601', '5755', '-1528.0', '18', '8', '--']
['08/19', '120010', '602', '5579', '-694.0', '20', '10', '--']


['08/19', '120103', '720', '4289', '-602.0', '20', '2', '19']
['08/19', '120103', '721', '5565', '-463.0', '30', '10', '21']
['08/19', '120103', '722', '6259', '-2685.0', '23', '8', '20']
['08/19', '120103', '723', '3964', '-926.0', '19', '2', '13']
['08/19', '120103', '724', '5956', '231.0', '51', '8', '22']
['08/19', '120103', '725', '3592', '-2963.0', '7', '2', '10']
['08/19', '120103', '726', '6944', '-972.0', '64', '8', '27']
['08/19', '120103', '727', '4492', '-1435.0', '28', '5', '16']
['08/19', '120103', '728', '5245', '-3009.0', '22', '5', '23']
['08/19', '120103', '729', '4376', '-694.0', '32', '4', '11']
['08/19', '120103', '730', '1857', '-1898.0', '2', '3', '6']
['08/19', '120103', '731', '6818', '3889.0', '71', '4', '24']
['08/19', '120103', '732', '1832', '-1343.0', '5', '2', '6']
['08/19', '120103', '733', '4197', '-2037.0', '19', '3', '14']
['08/19', '120103', '734', '7738', '-2639.0', '34', '7', '24']
['08/19', '120103', '735', '2660', '-556.0', '8', '3', '12']
['08/1

['08/19', '120088', '852', '1858', '-787.0', '3', '5', '8']
['08/19', '120088', '853', '882', '-741.0', '1', '4', '--']
['08/19', '120088', '854', '2061', '-1250.0', '5', '5', '1']
['08/19', '120088', '855', '674', '-139.0', '2', '0', '1']
['08/19', '120094', '856', '1018', '-278.0', '0', '2', '17']
['08/19', '120094', '857', '2281', '-1019.0', '0', '2', '28']
['08/19', '120094', '858', '2313', '231.0', '0', '2', '51']
['08/19', '120142', '859', '7685', '-1250.0', '0', '19', '61']
['08/19', '120142', '860', '8670', '12546.0', '0', '17', '117']
['08/19', '120142', '861', '9080', '-4630.0', '0', '21', '59']
['08/19', '120142', '862', '9318', '8611.0', '0', '22', '128']
['08/19', '120142', '863', '9546', '4722.0', '0', '23', '113']
['08/19', '120142', '864', '7736', '13426.0', '0', '19', '103']
['08/19', '120142', '865', '8274', '1204.0', '0', '19', '75']
['08/19', '120142', '866', '6694', '-324.0', '0', '14', '56']
['08/19', '120142', '867', '6469', '6435.0', '0', '13', '70']
['08/19', '

{'spreadsheetId': '1Z0Z1IDgbj-mGcRZxXEFpGlO00LHMQB_X5qll6QXUEOQ',
 'updates': {'spreadsheetId': '1Z0Z1IDgbj-mGcRZxXEFpGlO00LHMQB_X5qll6QXUEOQ',
  'updatedRange': 'S20RAW!F63973:M64332',
  'updatedRows': 360,
  'updatedColumns': 8,
  'updatedCells': 2880}}